In [14]:
import pandas as pd

dta_security=pd.read_csv("https://raw.githubusercontent.com/ucr-courses/knn-lab/refs/heads/main/data/jaggia_ba_1e_ch09_Security.csv")
dta_security.head()

,Threat,WarTerms,Keywords,Links
0,0,6,5,5
1,0,3,5,8
2,0,5,8,7
3,0,4,7,4
4,0,5,5,2


Let's keep 10% of data for testing and remaning for training/validation

In [15]:
from sklearn.model_selection import train_test_split
[X_train_val, X_test, y_train_val, y_test] = train_test_split(dta_security.drop("Threat",axis=1), dta_security["Threat"], test_size=0.4, random_state=42)

In [11]:
from re import X
y_train_val.head()

,MEDV
304,36.1
83,22.9
248,24.5
165,25.0
163,50.0


In [16]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

X_scaler=scaler.fit(X_train_val)
X_train_val_scaled=X_scaler.transform(X_train_val)
X_test_scaled=X_scaler.transform(X_test)

Setup up KNN model in scikit-learn

In [17]:
from sklearn.neighbors import KNeighborsClassifier

knn_model= KNeighborsClassifier()

Because we are interested in evaluating a variety of different possible values of $k$ in our model, we create a dictionary of values to explore using the label *n_neighbors* and a **range** of values between 1 and 10 (with the latter value being set to 10 + 1 since the end of **range** is exclusive). Enter:

In [18]:
paramGrid={'n_neighbors' : range( 1 , 10 + 1 )}

Now we are ready to fit our models and evaluate the different $k$ values. To do this, we will use $k$-fold cross-validation, where the $k$ in $k$-fold vs. the $k$ in KNN are not to be confused. Before we get started, we need to set up the procedure using the scikit-learn **GridSearchCV** function and store it in a variable with the label search. Within the function, the first two arguments are used to specify our model and our paramGrid from above; in addition, we use *cv* to set the number of folds to 10 and choose `'accuracy'` as our *scoring* metric to have results comparable to what is shown in the text (alternative scoring metrics may also be evaluated, which can be useful in various cases such as high class imbalance). Enter:

In [19]:
from sklearn.model_selection import GridSearchCV

search=GridSearchCV(knn_model,
                    paramGrid,
                    cv=10,
                    scoring="accuracy")

Now execute cross_validation

In [20]:
searchFit=search.fit(X_train_val_scaled,y_train_val)

Check results

In [ ]:
pd.DataFrame(searchFit.cv_results_)[['mean_test_score','params','rank_test_score']].sort_values(by='rank_test_score')

,mean_test_score,params,rank_test_score
5,0.655556,{'n_neighbors': 6},1
8,0.648148,{'n_neighbors': 9},2
9,0.640741,{'n_neighbors': 10},3
6,0.622222,{'n_neighbors': 7},4
4,0.622222,{'n_neighbors': 5},5
3,0.618519,{'n_neighbors': 4},6
2,0.607407,{'n_neighbors': 3},7
7,0.603704,{'n_neighbors': 8},8
0,0.577778,{'n_neighbors': 1},9
1,0.570370,{'n_neighbors': 2},10


The best performing k is ???

In [21]:
best_estimator=searchFit.best_estimator_
best_estimator

KNeighborsClassifier(n_neighbors=6)

We can either use searchFit to make predictions or use best_estimator to make predictions. searchFit uses the best estimator by default. Therefore, the results will be the same.

In [22]:
predicted_prob=searchFit.predict_proba(X_test_scaled)

In [23]:
predicted_prob=best_estimator.predict_proba(X_test_scaled)
predicted_prob

array([[0.66666667, 0.33333333],
       [0.16666667, 0.83333333],
       [0.16666667, 0.83333333],
       [0.33333333, 0.66666667],
       [0.33333333, 0.66666667],
       [0.        , 1.        ],
       [0.33333333, 0.66666667],
       [0.5       , 0.5       ],
       [0.        , 1.        ],
       [0.66666667, 0.33333333],
       [0.33333333, 0.66666667],
       [0.        , 1.        ],
       [0.66666667, 0.33333333],
       [0.        , 1.        ],
       [0.33333333, 0.66666667],
       [0.83333333, 0.16666667],
       [0.5       , 0.5       ],
       [0.83333333, 0.16666667],
       [0.        , 1.        ],
       [0.83333333, 0.16666667],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.66666667, 0.33333333],
       [0.66666667, 0.33333333],
       [0.16666667, 0.83333333],
       [0.16666667, 0.83333333],
       [0.33333333, 0.66666667],
       [0.5       , 0.5       ],
       [0.        , 1.        ],
       [0.5       , 0.5       ],
       [1.

The first column in the output above shows the probabilities of the cases belonging to Class 0 (Not threat), while the second column lists the probabilities of the cases belonging to Class 1 (threat).

To classify these probability predictions, let's use a threshold probability that reflects the proportion of 1s in our train-validation dataset. NOte that in python index starts from 0, therefore predicted_prob[:,1] gives us the **second** column.

In [25]:
cutoff_prob=0.5

In [28]:
# prompt: import numpy

import numpy as np

In [30]:
predicted_test=np.where(predicted_prob[:,1]>cutoff_prob,1,0)
predicted_test

array([0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 0])

In [31]:
from sklearn.metrics import confusion_matrix
# first argument is true values, second argument is predicted values
confusion_matrix(y_test,predicted_test)

array([[37, 15],
       [22, 46]])

The confusion matrix is arranged as follows

|                     | Predicted Negative | Predicted Positive |
|---------------------|--------------------|--------------------|
| **Actual Negative** | True Negatives (TN) | False Positives (FP) |
| **Actual Positive** | False Negatives (FN) | True Positives (TP) |


In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predicted_test)

0.6916666666666667

Alternatively we can use classification report for several other metrics

In [33]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predicted_test,target_names = ['Not threat', 'threat']))

              precision    recall  f1-score   support

  Not threat       0.63      0.71      0.67        52
      threat       0.75      0.68      0.71        68

    accuracy                           0.69       120
   macro avg       0.69      0.69      0.69       120
weighted avg       0.70      0.69      0.69       120



Optimal K = 6
Missclassification = 1-0.69 =0.31